Data Manipulation:
- path_to_model(file_name)
- data_call(dataset_name)
- determine_dataset(dataset,model_name) ... dataset in ['automatic','FRDEEP-F','MiraBest']

Model Manipulation:
- load_net(model_name,device)
- training_validation(PATH,xlims=[None,None],save=False,full_path=False) ... PATH is a local title of a folder or file (within ./TrainedNetworks)
- prediction(dataset, net, class_groups,(device='cuda',reps='360'))
- evaluate(file_name, dataset='automatic')

Evaluation Plots:
- plot_conf_mat(conf_matrix,normalised=True,n_classes=2,format_input=None,title='Confusion Matrix')
- plot_roc_curve(fpr,tpr,title='ROC Curve (AUC=\{auc:.3f\})')
- out_print(out)

Attention Maps:
- attentions_func(batch_of_images, net, mean=True, device=torch.device('cpu'))
- attention_analysis(source, source_only=True, attention_maps=None, GradCAM=None)
- AttentionImagesByEpoch(sources, folder_name, net,epoch=1500, device=torch.device('cpu'))
- attention_epoch_plot(source_images,folder_name, logged=False, width=3, device=torch.device('cpu'))

GradCAM:
- To be completed.

Other:
- mask_on_image(img, mask)
- SortedDataSamples(data_name, transformed=True,  rotations=1, subset='NOHYBRID')
- net_name_extraction(PATH)

Incomplete:
- Loading from Pickled dicts
- GradCAM Call for a given image

In [28]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F

import configparser as ConfigParser
import numpy as np

import utils
import networks

# Set seeds for reproduceability
torch.manual_seed(42)
np.radnom.seed(42)


class Model:
    def __init__(self, configfile):
        # Read in the config file
        self.config_name = configfile
        self.config = ConfigParser.SafeConfigParser(allow_no_value=True)
        self.config.read(self.config_name)
        
        self.n_classes = self.config['data']['num_classes']
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.net = utils.net.load(
            device=self.device, 
            **self.config['model'])
        self.data = utils.data.load(
            device=self.device, 
            rotations=self.config.getint('model', 'number_rotations'), 
            **self.config['data'])
        self.train = utils.train.load(self.config_name)
        
        self.model_trained = self.config.getboolean('grid_search', 'done')
        

    def model_trained(self):
        if self.config.getboolean('grid_search', 'done'):
            print('Training not required')
            #self.model = utils.load_model(**config['model'])
            #self.hyperparameters = self.config(**config['final_parameters'])
        else:
            print('Training required')
            #self.hyperparameters = utils.grid_search(self.data, **config['model'])
            #self.model = utils.train(**config['model'], self.data)


In [29]:
config_path = 'configs/template.cfg'
model = Model(config_path)

<ipython-input-28-9a044a9a200c>:14: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
  self.config = ConfigParser.SafeConfigParser(allow_no_value=True)


NameError: name 'config' is not defined

In [22]:
lr = model.config['grid_search']['learning_rate'].split(',')[1:]

False

In [6]:
config_path = f"configs/template.cfg"
config = configparser.ConfigParser()
config.read(config_path)
print(config.sections())
print(*config['model'])
print(config['model']['base'])

['model', 'data', 'training', 'output']
base nrot early_stopping quiet
'VanillaLeNet'            # network: [VanillaLeNet, CNSteerableLeNet, DNSteerableLeNet]
